In [1]:
import ray
import os
import time
from ray import tune
from pyhdfs import HdfsClient
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost_ray import RayXGBRFRegressor, RayParams
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

ray.init()

2022-11-22 21:10:39,972	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.6
Ray version:,2.1.0
Dashboard:,http://127.0.0.1:8265


In [2]:
nodes = ["127.0.0.1"]
client = HdfsClient(hosts=nodes, user_name="bigdata2022-VirtualBox")
df=pd.read_csv(client.open("hdfs://localhost:9870/user/bigdata2022/datasets/big_data_project/taxi_dataset_EDA_reduct.csv"))

In [3]:
df.head()
seed=42

In [4]:
X = df.drop(['trip_duration'], axis=1)
t = df['trip_duration']

In [5]:
X_train, X_test, t_train, t_test = train_test_split(X, t, train_size=0.75, random_state=seed)

In [6]:
start_time=time.time()

In [7]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
ray_param = RayParams(
                      num_actors=4,
                      cpus_per_actor=1
                     ) 

In [9]:
model = RayXGBRFRegressor(n_jobs=4, random_state=seed, max_depth= 3,
          min_child_weight= 0.3361,
          gamma= 14.96,
          learning_rate= 0.243,
          subsample= 0.53,
          colsample_bylevel= 1.0,
          colsample_bytree= 0.6099)

In [10]:
# param_grid = {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
#              'min_child_weight': np.arange(0.0001, 0.5, 0.001),
#              'gamma': np.arange(0.0,40.0,0.005),
#              'learning_rate': np.arange(0.0005,0.3,0.0005),
#              'subsample': np.arange(0.01,1.0,0.01),
#              'colsample_bylevel': np.round(np.arange(0.1,1.0,0.01)),
#              'colsample_bytree': np.arange(0.1,1.0,0.01)
#             }             

In [11]:
#grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring="r2")

In [12]:
#grid_result = grid_search.fit(X_train,t_train, ray_params=ray_param)

In [13]:
model.fit(X_train, t_train, ray_params=ray_param)#, eval_metric='r2')

/home/bigdata2022/.local/lib/python3.10/site-packages/xgboost/sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/bigdata2022/.local/lib/python3.10/site-packages/xgboost_ray/sklearn.py:307: UserWarning: `ray_params` is not `None` and will override the `n_jobs` attribute.
  warnings.warn("`ray_params` is not `None` and will override "
2022-11-22 21:10:43,780	INFO main.py:1035 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
2022-11-22 21:10:47,745	INFO main.py:1080 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=13282) [21:10:48] task [xgboost.ray]:139893552351696 got new rank 0
(_RemoteRayXGBoostActor pid=13283) [21:10:48] task [xgboost.ray]:140210882906576 got new rank 1
(_RemoteRayXGBoostActor pid=13336) [21:10:48] task [xgboost.ray]:139869993896352 got new rank 2
(_Re

RuntimeError: A Ray actor died during training and the maximum number of retries (0) is exhausted.

In [ ]:
pred_ray = model.predict(X_test)
print(pred_ray)

In [ ]:
params = model.get_xgb_params()
print(params)

In [ ]:
print(time.time()-start_time)

In [ ]:
#save_model("model.xgb")